In [47]:
#PREDICTING THE REVENUE OF THE ADVERTISEMENTS(ad's).

In [48]:
#importing required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

In [49]:
#readig the previous data as train_data
train_data=pd.read_csv(r"C:\Users\sailo\Downloads\Train_Data.csv")

In [50]:
#reading the data to be estimated as test_data
test_data=pd.read_csv(r"C:\Users\sailo\Downloads\Test_Data.csv")

In [51]:
#checking first few data of train_data
train_data.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71


In [52]:
#checking first few data of test_data
test_data.head()

,date,campaign,adgroup,ad,cost,impressions,clicks,conversions
0,01-03-2021,campaign 1,adgroup 1,ad 1,0.58,121,49,1
1,01-03-2021,campaign 1,adgroup 3,ad 1,0.17,22,12,0
2,01-03-2021,campaign 1,adgroup 4,ad 1,0.05,5,3,0
3,01-03-2021,campaign 1,adgroup 2,ad 1,0.01,2,1,0
4,01-03-2021,campaign 1,adgroup 2,ad 2,0.01,3,1,0


In [53]:
#checking the null points in data
train_data.isnull().sum().max()

0

In [54]:
#checking the null points in data
test_data.isnull().sum().max()

0

In [55]:
#for training the train_data spliting into X and Y

In [56]:
#removing the 'revenue' data which we need to predict from the train_data and removing the 'date' data which is of no-use in predicting and 
#assigning this data in X 
X=train_data.drop(['revenue','date'],axis=1)
#assigning the revenue data as Y
Y=train_data.revenue
#Assigning the test_data as x_test_full
x_test_full=test_data.drop(['date'],axis=1)

In [57]:
#dividing data for training the model into X_train_full,y_train full for traing and x_valid_full,y_valid_full for checking the predictive test 
#in the train_data
x_train_full,x_valid_full,y_train,y_valid=train_test_split(X,Y,random_state=10)

In [58]:
#diving the data into catagorical and numerical data columns for encoding the catagorial data
#x_cata_col for catagorical data columns and for getting better prediction we are taking the no.of didtinct data as less than 10(x_train_full[i].nunique() < 10 )
x_cat_col= [i for i in x_train_full.columns if
                    x_train_full[i].nunique() < 10 and
                    x_train_full[i].dtype == "object"]
x_cat_col

['campaign', 'adgroup']

In [59]:
#x_num_col for numerical data columns
x_num_col=[i for i in x_train_full.columns if 
                x_train_full[i].dtype in ['int64', 'float64']]
x_num_col

['impressions', 'clicks', 'cost', 'conversions']

In [60]:
#t_col as total columns i.e,catogrical columns and numerical columns after feature engineering
t_col=x_cat_col+x_num_col
t_col

['campaign', 'adgroup', 'impressions', 'clicks', 'cost', 'conversions']

In [61]:
#new x_train,x_valid,x_test after modifing the data columns
x_train=x_train_full[t_col].copy()
x_valid = x_valid_full[t_col].copy()
test_new = x_test_full[t_col].copy()

In [62]:
#preprocessing the data by using the simple imputer for numerical data to replace the NANN values in the data and using the piplining in 
#catagorical data for assigning the steps of imputing the NANN values and encoding the catagorical data.

In [63]:
numerical_transformer = SimpleImputer(strategy='most_frequent')
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, x_num_col),('cat', categorical_transformer, x_cat_col) ])

In [64]:
#using the RandomForestRegressor as machine learning model
model = RandomForestRegressor(n_estimators=100, random_state=10)

In [65]:
#using the pipline for piplining the steps for the other data
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [66]:
#fitting the train_data
clf.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['impressions', 'clicks',
                                                   'cost', 'conversions']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['campaign', 'adgroup'])])),
                ('model', RandomForestRegressor(random_state=10))])

In [67]:
#checking the mean_absolute_error for the x_valid data predicted revenue data and original revenue data
mean_absolute_error(y_valid,clf.predict(x_valid))

77.05907177019543

In [68]:
#checking the cross validatin score
from sklearn.model_selection import cross_val_score
cross_val_score(clf,x_valid,y_valid,cv=5)

array([0.86394199, 0.44530937, 0.89732414, 0.88902155, 0.88208772])

In [69]:
#precting the test data revenue
predictions=clf.predict(test_new)

In [70]:
#saving the revenue data locally
res = pd.DataFrame(predictions) 
res.index = test_new.index 
res.columns = ["revenue"]
res.to_csv("prediction_results.csv", index = False)      